In [208]:
import numpy as np
import pandas as pd
shanghai_ranking = pd.read_csv('data/Shanghai_ranking_data_2023.csv')
THE_ranking = pd.read_csv('data/THE_ranking_data_2023.csv')
qs_ranking = pd.read_csv('data/QS_ranking_data_2023.csv')

In [209]:
def merge(Rank1, Rank2, bias):
    import Levenshtein
    distance = np.empty((len(Rank1.Name),len(Rank2.Name)))
    Rank1 = Rank1.rename(columns={'Name':'Name1'})
    Rank2 = Rank2.rename(columns={'Name':'Name2'})
    for index1, rank1 in enumerate(Rank1.Name1):
        for index2, rank2 in enumerate(Rank2.Name2):
            distance[index1, index2] = Levenshtein.distance(rank1, rank2)
    merged_rank = pd.DataFrame({})
    for i in range(distance.shape[0]):
        j = np.argmin(distance[i, :])
        if distance[i, j] == bias:
            # print('i:',i,'j:',j)
            merged_row = pd.concat([Rank1.iloc[i], Rank2.iloc[j]], axis=0)
            merged_row = merged_row.to_frame().T
            merged_rank = pd.concat([merged_rank, merged_row])
    merged_rank = merged_rank.reset_index(drop=True)
    return merged_rank

def clean(merged_rank, row=[]):
    for i in row:
        merged_rank = merged_rank.drop(i)
    merged_rank = merged_rank.drop('Name2',axis=1)
    merged_rank = merged_rank.rename(columns={'Name1':'Name'}).reset_index(drop=True)
    return merged_rank

In [223]:
merged_rank1 = merge(shanghai_ranking,qs_ranking,0)
merged_rank1 = clean(merged_rank1)
merged_rank2 = merge(shanghai_ranking,qs_ranking,1)
merged_rank2 = clean(merged_rank2)
merged_rank = pd.concat([merged_rank1,merged_rank2])
merged_rank

,Rank,Name,Region,Regional rank,Total score,Alumni,Award,Hici,N&S,PUB,...,cpf rank,ifr score,ifr rank,isr score,isr rank,irn score,irn rank,ger score,ger rank,score scaled
0,1,Harvard University,us,1,100.0,100.0,92.6,100.0,100.0,100.0,...,2,76.9,228,66.9,212,100.0,1,100.0,1,97.6
1,2,Stanford University,us,2,74.8,45.6,88.1,73.6,82.8,67.5,...,9,99.8,74,60.3,235,96.3,55,100.0,2,98.5
2,4,University of Cambridge,gb,1,67.9,78.9,90.8,45.1,61.0,63.6,...,55,100.0,60,96.3,70,99.5,6,100.0,9,98.8
3,6,Princeton University,us,5,60.1,61.7,100.0,32.9,50.4,39.4,...,3,26.0,515,62.2,226,78.5,340,100.0,8,89.2
4,7,University of Oxford,gb,2,59.5,49.9,59.7,51.4,57.6,71.4,...,64,98.8,101,98.4,54,99.9,3,100.0,7,98.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,901-1000,Gdansk University of Technology,pl,5-9,NaN,12.9,0.0,6.6,0.0,20.5,...,601+,3.3,601+,5.3,601+,61.8,548,6.6,601+,-
48,901-1000,Nicolaus Copernicus University,pl,5-9,NaN,10.1,0.0,0.0,5.2,23.0,...,601+,3.6,601+,3.9,601+,71.5,441,5.2,601+,-
49,901-1000,Southern Cross University,au,31-34,NaN,21.9,0.0,0.0,7.3,15.1,...,393,73.5,243,88.1,120,57.3,601+,7.1,601+,-
50,901-1000,Tokyo University of Agriculture and Technology,jp,29-32,NaN,15.8,0.0,0.0,6.9,17.3,...,335,5.6,601+,6.4,601+,46.2,601+,5.1,601+,-


In [230]:
merged_rank1 = merge(merged_rank,THE_ranking,0)
merged_rank1 = clean(merged_rank1)
merged_rank2 = merge(merged_rank,THE_ranking,1)
merged_rank2 = clean(merged_rank2)
merged_rank = pd.concat([merged_rank1,merged_rank2])
merged_rank

,Rank,Name,Region,Regional rank,Total score,Alumni,Award,Hici,N&S,PUB,...,No. of FTE students,No. of students per staff,International Students,Female:Male Ratio,Overall,Teaching,Research Environment,Research Quality,Industry,International Outlook
0,1,Harvard University,us,1,100.0,100.0,92.6,100.0,100.0,100.0,...,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
1,2,Stanford University,us,2,74.8,45.6,88.1,73.6,82.8,67.5,...,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
2,4,University of Cambridge,gb,1,67.9,78.9,90.8,45.1,61.0,63.6,...,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,6,Princeton University,us,5,60.1,61.7,100.0,32.9,50.4,39.4,...,"8,279",8.0,23%,46 : 54,92.4,87.6,95.9,99.1,66.0,80.3
4,7,University of Oxford,gb,2,59.5,49.9,59.7,51.4,57.6,71.4,...,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,101-150,Xi'an Jiaotong University,cn,11-23,NaN,20.3,0.0,18.6,16.4,70.6,...,"44,942",14.1,6%,35 : 65,39.3–42.0,36.4,42.7,47.1,75.0,29.4
5,151-200,Aix Marseille University,fr,6-8,NaN,19.0,0.0,18.6,21.5,48.1,...,"70,054",17.4,13%,60 : 40,47.0–48.7,40.5,24.9,72.3,41.5,63.2
6,301-400,Linkoping University,se,7-8,NaN,19.1,0.0,13.2,10.6,32.0,...,"15,026",14.4,15%,52 : 48,48.9–51.1,23.5,39.8,78.0,61.8,70.8
7,301-400,University of Milano-Bicocca,it,9-10,NaN,23.1,0.0,11.4,7.2,34.2,...,"29,057",29.2,5%,62 : 38,47.0–48.7,24.9,25.9,93.6,40.8,43.0
